In [1]:
import numpy as np
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import json
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import os
from collections import defaultdict


def load_json_data(file_path):
    """
    加载JSON文件数据
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

def split_by_source_random(data, num_groups):
    """
    根据来源(source)将数据随机分成num_groups份    
    Args:
        data: JSON数据列表
        num_groups: 需要分成的组数
    Returns:
        分组后的数据字典，键为组号，值为该组的数据列表
    """
    # 按source分组
    source_groups = defaultdict(list)
    for item in data:
        source = item.get('source', 'Unknown')
        source_groups[source].append(item)
    
    # 创建空组
    result_groups = {i: [] for i in range(num_groups)}
    
    # 获取所有来源并随机排序
    sources = list(source_groups.keys())
    random.shuffle(sources)
    
    # 为每个来源分配一个组
    for i, source in enumerate(sources):
        group_idx = i % num_groups  # 循环分配来源到各组
        result_groups[group_idx].extend(source_groups[source])
    
    return result_groups

import random
from collections import defaultdict

def split_by_source_balanced(data, num_groups):
    """
    根据来源(source)将数据均衡地分配到num_groups个组中
    
    Args:
        data: JSON数据列表，每个元素需包含'source'字段
        num_groups: 需要分成的组数
    
    Returns:
        dict: 键为组号(0~num_groups-1)，值为该组的数据列表
    """
    # 1. 按source分组并统计每个来源的数据量
    source_groups = defaultdict(list)
    source_counts = defaultdict(int)
    for item in data:
        source = item.get('source', 'Unknown')
        source_groups[source].append(item)
        source_counts[source] += 1

    # 2. 按数据量从大到小排序来源（优先分配大数据量来源）
    sorted_sources = sorted(source_counts.keys(), 
                           key=lambda x: -source_counts[x])

    # 3. 初始化各组（记录当前组的数据量）
    result_groups = {i: [] for i in range(num_groups)}
    group_counts = [0] * num_groups  # 记录各组当前数据量

    # 4. 贪心算法分配：总是将当前来源分配给数据量最少的组
    for source in sorted_sources:
        # 找到当前数据量最少的组
        target_group = min(range(num_groups), key=lambda x: group_counts[x])
        # 将整个来源的数据分配到该组
        result_groups[target_group].extend(source_groups[source])
        group_counts[target_group] += source_counts[source]

    # 5. 打印各组数量分布（调试用）
    print("各组数据量分布:", group_counts)
    
    return result_groups

def save_split_data(groups, output_dir):
    """
    将分组数据保存到文件
    Args:
        groups: 分组后的数据字典
        output_dir: 输出目录
    """
    os.makedirs(output_dir, exist_ok=True)
    
    for group_idx, group_data in groups.items():
        output_path = os.path.join(output_dir, f'group_{group_idx}.json')
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(group_data, f, ensure_ascii=False, indent=2)
        print(f"保存组 {group_idx} 到 {output_path}，包含 {len(group_data)} 条数据")

def split_by_clustering(data, num_groups):
    """
    todo: 效果很差，还没找到一个聚类效果好的办法
    使用聚类方法将数据分成num_groups份 
    Args:
        data: JSON数据列表
        num_groups: 需要分成的组数
    Returns:
        分组后的数据字典，键为组号，值为该组的数据列表
    """

In [2]:
# 设置文件路径
input_file = 'dataset/corpus.json'  # 替换为你的JSON文件路径

# 加载数据
data = load_json_data(input_file)
print(f"加载了 {len(data)} 条数据")

# 示例1：按source均匀分成3组
groups_by_source = split_by_source_balanced(data, 3)
save_split_data(groups_by_source, 'splits/balance_3')

加载了 609 条数据
各组数据量分布: [203, 203, 203]
保存组 0 到 splits/balance_3\group_0.json，包含 203 条数据
保存组 1 到 splits/balance_3\group_1.json，包含 203 条数据
保存组 2 到 splits/balance_3\group_2.json，包含 203 条数据
